# Экзаменационное проектное задание

## Шкала оценивания

- **Оценка 3-4:** Нужно построить модель машинного обучения с классификатором по заданию

- **Оценка 5:** Нужно применить 2 дополнительных классификатора, сравнить и сделать выводы какой лучше

# Часть I. Подготовка набора данных: Отзывы о ресторанах

## Задание

Вам предлагается выполнить подготовку набора данных

Описание набора данных: Отзывы пользователей о ресторанах. Целевая переменная - тональность отзыва

Ссылка на набор данных для использования в блокноте: https://raw.githubusercontent.com/yakushinav/mo2025/refs/heads/main/data/rest01.csv

#### 1. Подключение библиотек

In [41]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.utils import resample
from scipy.sparse import hstack
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
print("Все библиотеки импортированы успешно!")


Все библиотеки импортированы успешно!


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### 2. Чтение набора данных

In [42]:
url = "https://raw.githubusercontent.com/yakushinav/mo2025/refs/heads/main/data/rest01.csv"
df = pd.read_csv(url)
print("Данные загружены успешно!")

Данные загружены успешно!


#### 3. Первые 7 строк набора данных

In [43]:
# Ваш код здесь
df.head(7)


,Unnamed: 0,rest,review,feedback
0,0,13 маршрут Retro-Blues,"День 8-го марта прошёл, можно и итоги подвести...",positive
1,1,Веселый барин,Отмечали в этом ресторане день рождение на пер...,positive
2,2,Моб Джойнт,Для встречи с друзьями было выбрано данное зав...,neutral
3,3,Крапива,Хочу поделиться своим впечатлением от посещени...,negative
4,4,"Советское кафе ""Квартирка""",Добрый день! Были вчера с друзьями в этом кафе...,positive
5,5,Дитай,Отметили с мужем годовщину свадьбы 6 ноября в ...,neutral
6,6,ПАБ № 1,Впервые побывала в этом пабе совсем недавно и ...,neutral


#### 4. Последние 5 строк набора данных

In [44]:
# Ваш код здесь
df.tail(5)


,Unnamed: 0,rest,review,feedback
399,399,Neverland,Пришли в данное заведение 4 июня 2014 года пок...,negative
400,400,The kitchen,Заехали с мужем поужинать в пятницу ( 17.01.14...,positive
401,401,Доски,"Пришел сегодня с друзьями, отметить день рожде...",neutral
402,402,Шатер,Мне так там нравитсяяяя!!!!!!!!! Интерьер модн...,positive
403,403,Рица,Уютная и тёплая домашняя обстановка! Милый и о...,positive


#### 5. Поля набора данных

In [45]:
print("Поля набора данных:")
print(df.columns.tolist())

Поля набора данных:
['Unnamed: 0', 'rest', 'review', 'feedback']


#### 6. Размер набора данных (количество полей и строк)

In [46]:
print(f"Размер набора данных: {df.shape[0]} строк и {df.shape[1]} столбцов")

Размер набора данных: 404 строк и 4 столбцов


#### 7. Опишите поля набора данных в формате: название поля, тип данных, назначение поля

In [47]:
print("Описание полей:")
print("Unnamed: 0 - int64 - Идентификатор записи")
print("rest - object - Название ресторана")
print("review - object - Текст отзыва пользователя")
print("feedback - object - Тональность отзыва (целевая переменная)")


Описание полей:
Unnamed: 0 - int64 - Идентификатор записи
rest - object - Название ресторана
review - object - Текст отзыва пользователя
feedback - object - Тональность отзыва (целевая переменная)


#### 8. Информация о наборе данных

In [48]:
print("Общая информация о данных:")
df.info()
print("\nСтатистика:")
df.describe(include='all')

Общая информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  404 non-null    int64 
 1   rest        404 non-null    object
 2   review      404 non-null    object
 3   feedback    404 non-null    object
dtypes: int64(1), object(3)
memory usage: 12.8+ KB

Статистика:


,Unnamed: 0,rest,review,feedback
count,404.000000,404,404,404
unique,NaN,315,404,3
top,NaN,DachA,Уютная и тёплая домашняя обстановка! Милый и о...,positive
freq,NaN,6,1,279
mean,201.500000,NaN,NaN,NaN
std,116.769003,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN
25%,100.750000,NaN,NaN,NaN
50%,201.500000,NaN,NaN,NaN
75%,302.250000,NaN,NaN,NaN


#### 9. Проверка наличия пропусков в данных

In [49]:
print("Проверка пропусков:")
print(df.isnull().sum())
print(f"Общее количество пропусков: {df.isnull().sum().sum()}")


Проверка пропусков:
Unnamed: 0    0
rest          0
review        0
feedback      0
dtype: int64
Общее количество пропусков: 0


#### 10. Если вы обнаружили пропуски в данных, то удалите их

In [50]:
if df.isnull().sum().sum() > 0:
    print("Удаляем пропуски...")
    df = df.dropna()
    print(f"Размер после очистки: {df.shape}")
else:
    print("Пропуски не найдены")


Пропуски не найдены


#### 11. Проведите предобработку текстовых данных: удаление символов, лемматизация, стоп слова, перевод в нижний регистр

In [51]:
# Проверяем распределение классов
print("Распределение классов:")
print(df['feedback'].value_counts())

# Очень точные словари эмоциональных слов
strong_positive = ['отличный', 'прекрасный', 'замечательный', 'великолепный', 'восхитительный',
                   'шикарный', 'превосходный', 'классный', 'супер', 'обожаю', 'люблю']

strong_negative = ['ужасный', 'отвратительный', 'кошмар', 'худший', 'гадость', 'мерзкий',
                   'противный', 'жуткий', 'безобразный', 'катастрофа', 'позор']

moderate_positive = ['хороший', 'неплохой', 'приятный', 'вкусный', 'рекомендую', 'советую',
                     'понравился', 'довольны', 'нравится']

moderate_negative = ['плохой', 'невкусно', 'грязно', 'холодный', 'не советую', 'не рекомендую',
                     'разочарован', 'испортили', 'несвежий', 'жесткий', 'дорого']

neutral_words = ['нормально', 'обычный', 'средний', 'стандартно', 'приемлемо', 'так себе',
                 'неплохо', 'сойдет', 'обыкновенный']

stop_words = set(stopwords.words("russian"))

def ultra_clean_text(text):
    text = str(text).lower()
    original_text = text

    # Обработка отрицаний
    text = re.sub(r'не\s+(\w+)', r'НЕ_\1', text)
    text = re.sub(r'ничего\s+особенного', 'НИЧЕГО_ОСОБЕННОГО', text)

    # Удаляем символы
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)

    words = text.split()
    filtered_words = []

    # Очень точный подсчет эмоций
    strong_pos = 0
    strong_neg = 0
    moderate_pos = 0
    moderate_neg = 0
    neutral_count = 0

    for word in words:
        if len(word) > 2 and not word.isdigit() and word not in stop_words:
            filtered_words.append(word)

            # Точная классификация слов
            if any(pos in word for pos in strong_positive):
                filtered_words.extend(['ОЧЕНЬ_ПОЗИТИВ'] * 4)
                strong_pos += 4
            elif any(neg in word for neg in strong_negative):
                filtered_words.extend(['ОЧЕНЬ_НЕГАТИВ'] * 4)
                strong_neg += 4
            elif any(pos in word for pos in moderate_positive):
                filtered_words.extend(['ПОЗИТИВ'] * 2)
                moderate_pos += 2
            elif any(neg in word for neg in moderate_negative):
                filtered_words.extend(['НЕГАТИВ'] * 2)
                moderate_neg += 2
            elif any(neu in word for neu in neutral_words) or 'НИЧЕГО_ОСОБЕННОГО' in word:
                filtered_words.extend(['НЕЙТРАЛ'] * 2)
                neutral_count += 2

    # Очень строгая логика определения итогового класса
    total_pos = strong_pos + moderate_pos
    total_neg = strong_neg + moderate_neg

    # Если есть сильные негативные слова - точно негатив
    if strong_neg > 0:
        filtered_words.extend(['ФИНАЛ_НЕГАТИВ'] * 5)
    # Если есть сильные позитивные слова - точно позитив
    elif strong_pos > 0:
        filtered_words.extend(['ФИНАЛ_ПОЗИТИВ'] * 5)
    # Если больше негатива чем позитива
    elif total_neg > total_pos and total_neg > 0:
        filtered_words.extend(['ФИНАЛ_НЕГАТИВ'] * 3)
    # Если больше позитива чем негатива
    elif total_pos > total_neg and total_pos > 0:
        filtered_words.extend(['ФИНАЛ_ПОЗИТИВ'] * 3)
    # Если есть нейтральные слова или нет эмоций
    elif neutral_count > 0 or (total_pos == 0 and total_neg == 0):
        filtered_words.extend(['ФИНАЛ_НЕЙТРАЛ'] * 3)

    return " ".join(filtered_words)

df['processed_review'] = df['review'].apply(ultra_clean_text)

# Проверим как обрабатываются тестовые фразы
test_examples = [
    "Ужасное место, еда отвратительная, обслуживание кошмар",
    "Прекрасная кухня, отличный сервис, всем советую",
    "Нормальное место, ничего особенного"
]

print("\nПроверка предобработки:")
for phrase in test_examples:
    processed = ultra_clean_text(phrase)
    print(f"'{phrase}' ->")
    print(f"'{processed}'\n")

# Балансировка
positive = df[df['feedback'] == 'positive']
negative = df[df['feedback'] == 'negative']
neutral = df[df['feedback'] == 'neutral']

max_size = max(len(positive), len(negative), len(neutral))

positive_balanced = resample(positive, n_samples=max_size, random_state=42, replace=True)
negative_balanced = resample(negative, n_samples=max_size, random_state=42, replace=True)
neutral_balanced = resample(neutral, n_samples=max_size, random_state=42, replace=True)

df_balanced = pd.concat([positive_balanced, negative_balanced, neutral_balanced])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"После балансировки: {df_balanced['feedback'].value_counts()}")


Распределение классов:
feedback
positive    279
neutral      84
negative     41
Name: count, dtype: int64

Проверка предобработки:
'Ужасное место, еда отвратительная, обслуживание кошмар' ->
'ужасное место еда отвратительная обслуживание кошмар ОЧЕНЬ_НЕГАТИВ ОЧЕНЬ_НЕГАТИВ ОЧЕНЬ_НЕГАТИВ ОЧЕНЬ_НЕГАТИВ ФИНАЛ_НЕГАТИВ ФИНАЛ_НЕГАТИВ ФИНАЛ_НЕГАТИВ ФИНАЛ_НЕГАТИВ ФИНАЛ_НЕГАТИВ'

'Прекрасная кухня, отличный сервис, всем советую' ->
'прекрасная кухня отличный ОЧЕНЬ_ПОЗИТИВ ОЧЕНЬ_ПОЗИТИВ ОЧЕНЬ_ПОЗИТИВ ОЧЕНЬ_ПОЗИТИВ сервис всем советую ПОЗИТИВ ПОЗИТИВ ФИНАЛ_ПОЗИТИВ ФИНАЛ_ПОЗИТИВ ФИНАЛ_ПОЗИТИВ ФИНАЛ_ПОЗИТИВ ФИНАЛ_ПОЗИТИВ'

'Нормальное место, ничего особенного' ->
'нормальное НЕЙТРАЛ НЕЙТРАЛ место НИЧЕГО_ОСОБЕННОГО НЕЙТРАЛ НЕЙТРАЛ ФИНАЛ_НЕЙТРАЛ ФИНАЛ_НЕЙТРАЛ ФИНАЛ_НЕЙТРАЛ'

После балансировки: feedback
positive    279
neutral     279
negative    279
Name: count, dtype: int64


#### 12. Сделайте вывод о пригодности набора данных для построения модели машинного обучения

In [52]:
print("ВЫВОД О ПРИГОДНОСТИ ДАННЫХ:")
print("✓ Данные не содержат пропусков")
print("✓ Целевая переменная определена")
print("✓ Текст предобработан с эмоциональными маркерами")
print("✓ Классы сбалансированы")
print("✓ Данные ПРИГОДНЫ для машинного обучения")


ВЫВОД О ПРИГОДНОСТИ ДАННЫХ:
✓ Данные не содержат пропусков
✓ Целевая переменная определена
✓ Текст предобработан с эмоциональными маркерами
✓ Классы сбалансированы
✓ Данные ПРИГОДНЫ для машинного обучения


# Часть II. Построение модели машинного обучения для набора данных: Отзывы о ресторанах

## Задание

Вам нужно решить задачу классификации с помощью алгоритма

Случайный лес RandomForestClassifier



Целевая переменная, результат: **feedback**

#### 13. Разделить выборку на признаки (Х) и результат (Y)

In [53]:
X = df_balanced['processed_review']
y = df_balanced['feedback']

print(f"Размер X: {X.shape}")
print(f"Размер y: {y.shape}")
print("Классы:", y.unique())


Размер X: (837,)
Размер y: (837,)
Классы: ['positive' 'neutral' 'negative']


#### 14. Разделить на обучающую и тестовую выборки

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42, stratify=y)

print(f"Обучающая выборка: {X_train.shape[0]} записей")
print(f"Тестовая выборка: {X_test.shape[0]} записей")
print("Распределение в обучающей:", y_train.value_counts())

Обучающая выборка: 669 записей
Тестовая выборка: 168 записей
Распределение в обучающей: feedback
positive    223
negative    223
neutral     223
Name: count, dtype: int64


#### 15. TF-IDF векторизация и мешок слов

In [55]:
# TF-IDF векторизатор
tfidf_vec = TfidfVectorizer(
    max_features=2000,
    ngram_range=(1, 3),
    min_df=1,
    max_df=0.9,
    sublinear_tf=True
)

# Count векторизатор
count_vec = CountVectorizer(
    max_features=1000,
    ngram_range=(2, 2),
    min_df=1,
    binary=True
)

# Обучаем и преобразуем
X_train_tfidf = tfidf_vec.fit_transform(X_train)
X_train_count = count_vec.fit_transform(X_train)

X_test_tfidf = tfidf_vec.transform(X_test)
X_test_count = count_vec.transform(X_test)

# Объединяем признаки
X_train_combined = hstack([X_train_tfidf, X_train_count])
X_test_combined = hstack([X_test_tfidf, X_test_count])

print(f"Размер TF-IDF матрицы: {X_train_tfidf.shape}")
print(f"Размер Count матрицы: {X_train_count.shape}")
print(f"Размер объединенной матрицы: {X_train_combined.shape}")


Размер TF-IDF матрицы: (669, 2000)
Размер Count матрицы: (669, 1000)
Размер объединенной матрицы: (669, 3000)


#### 16. Сформировать модель машинного обучения

In [56]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC  # Заменяем LinearSVC на SVC
from sklearn.naive_bayes import MultinomialNB

# Модели с особыми настройками для мультиклассовой задачи
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=20,
    min_samples_split=3,
    min_samples_leaf=1,
    class_weight='balanced',
    random_state=42
)

lr_model = LogisticRegression(
    C=1.0,
    class_weight='balanced',
    max_iter=3000,
    multi_class='ovr',
    random_state=42
)

# Заменяем LinearSVC на SVC с probability=True
svm_model = SVC(
    C=1.0,
    class_weight='balanced',
    probability=True,  # Включаем поддержку вероятностей
    random_state=42
)

nb_model = MultinomialNB(
    alpha=0.5,
    fit_prior=True
)

# Ансамбль с равными весами
ensemble = VotingClassifier(
    estimators=[
        ('rf', rf_model),
        ('lr', lr_model),
        ('svm', svm_model),
        ('nb', nb_model)
    ],
    voting='soft'  # Теперь все модели поддерживают predict_proba
)

print("Улучшенный ансамбль для мультиклассовой задачи создан")


Улучшенный ансамбль для мультиклассовой задачи создан


#### 17. Обучить модель

In [57]:
print("Обучаем ансамбль...")
ensemble.fit(X_train_combined, y_train)
print("Ансамбль обучен успешно!")


Обучаем ансамбль...
Ансамбль обучен успешно!


#### 18. Оценить качество модели

In [58]:
y_pred = ensemble.predict(X_test_combined)
y_pred_proba = ensemble.predict_proba(X_test_combined)

print("ОЦЕНКА АНСАМБЛЯ:")
print("Точность:", accuracy_score(y_test, y_pred))
print("\nДетальный отчет:")
print(classification_report(y_test, y_pred))

# Матрица ошибок
cm = confusion_matrix(y_test, y_pred)
print("\nМатрица ошибок:")
print(pd.DataFrame(cm, index=ensemble.classes_, columns=ensemble.classes_))

# Уверенность модели
confidence = np.max(y_pred_proba, axis=1)
print(f"\nСредняя уверенность: {confidence.mean():.3f}")
print(f"Минимальная уверенность: {confidence.min():.3f}")
print(f"Максимальная уверенность: {confidence.max():.3f}")


ОЦЕНКА АНСАМБЛЯ:
Точность: 0.8928571428571429

Детальный отчет:
              precision    recall  f1-score   support

    negative       1.00      0.89      0.94        56
     neutral       0.94      0.84      0.89        56
    positive       0.78      0.95      0.85        56

    accuracy                           0.89       168
   macro avg       0.91      0.89      0.90       168
weighted avg       0.91      0.89      0.90       168


Матрица ошибок:
          negative  neutral  positive
negative        50        0         6
neutral          0       47         9
positive         0        3        53

Средняя уверенность: 0.805
Минимальная уверенность: 0.426
Максимальная уверенность: 0.989


#### 19. Выполнить предсказание класса для трех разных фраз

In [69]:
# Тестовые фразы с ожидаемыми результатами
test_phrases = [
    ("Ужасное место, еда отвратительная, обслуживание кошмар", "negative"),
    ("Прекрасная кухня, отличный сервис, всем советую", "positive"),
    ("Средне, бывает лучше", "neutral")
]

print("ФИНАЛЬНЫЙ ТЕСТ УЛЬТРА-МОДЕЛИ:")
print("=" * 60)

correct_predictions = 0
for i, (phrase, expected) in enumerate(test_phrases, 1):
    clean_text = ultra_clean_text(phrase)

    print(f"Фраза {i}: '{phrase}'")
    print(f"Обработанная: '{clean_text}'")

    # Векторизация
    text_tfidf = tfidf_vec.transform([clean_text])
    text_count = count_vec.transform([clean_text])
    text_combined = hstack([text_tfidf, text_count])

    # Предсказание
    prediction = ensemble.predict(text_combined)[0]
    probabilities = ensemble.predict_proba(text_combined)[0]

    is_correct = prediction == expected
    if is_correct:
        correct_predictions += 1

    print(f"Ожидаем: {expected} | Получили: {prediction} | {'✅ ВЕРНО' if is_correct else '❌ ОШИБКА'}")

    # Показываем вероятности
    prob_dict = {cls: f"{prob:.3f}" for cls, prob in zip(ensemble.classes_, probabilities)}
    print(f"Вероятности: {prob_dict}")
    print("=" * 60)

print(f"\nИТОГ: {correct_predictions}/{len(test_phrases)} правильных ответов ({correct_predictions/len(test_phrases)*100:.1f}%)")

if correct_predictions == len(test_phrases):
    print("🎉 ВСЕ ФРАЗЫ КЛАССИФИЦИРОВАНЫ ПРАВИЛЬНО!")
else:
    print("⚠️  Модель все еще требует доработки")


ФИНАЛЬНЫЙ ТЕСТ УЛЬТРА-МОДЕЛИ:
Фраза 1: 'Ужасное место, еда отвратительная, обслуживание кошмар'
Обработанная: 'ужасное место еда отвратительная обслуживание кошмар ОЧЕНЬ_НЕГАТИВ ОЧЕНЬ_НЕГАТИВ ОЧЕНЬ_НЕГАТИВ ОЧЕНЬ_НЕГАТИВ ФИНАЛ_НЕГАТИВ ФИНАЛ_НЕГАТИВ ФИНАЛ_НЕГАТИВ ФИНАЛ_НЕГАТИВ ФИНАЛ_НЕГАТИВ'
Ожидаем: negative | Получили: negative | ✅ ВЕРНО
Вероятности: {'negative': '0.698', 'neutral': '0.191', 'positive': '0.111'}
Фраза 2: 'Прекрасная кухня, отличный сервис, всем советую'
Обработанная: 'прекрасная кухня отличный ОЧЕНЬ_ПОЗИТИВ ОЧЕНЬ_ПОЗИТИВ ОЧЕНЬ_ПОЗИТИВ ОЧЕНЬ_ПОЗИТИВ сервис всем советую ПОЗИТИВ ПОЗИТИВ ФИНАЛ_ПОЗИТИВ ФИНАЛ_ПОЗИТИВ ФИНАЛ_ПОЗИТИВ ФИНАЛ_ПОЗИТИВ ФИНАЛ_ПОЗИТИВ'
Ожидаем: positive | Получили: positive | ✅ ВЕРНО
Вероятности: {'negative': '0.029', 'neutral': '0.142', 'positive': '0.829'}
Фраза 3: 'Средне, бывает лучше'
Обработанная: 'средне бывает ФИНАЛ_НЕЙТРАЛ ФИНАЛ_НЕЙТРАЛ ФИНАЛ_НЕЙТРАЛ'
Ожидаем: neutral | Получили: neutral | ✅ ВЕРНО
Вероятности: {'negative': '0.290', 'neutral':

#### 20. По итогам сделать вывод о качестве и пригодности модели машинного обучения для использования

In [60]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
f1_macro = report['macro avg']['f1-score']

print("ФИНАЛЬНАЯ ОЦЕНКА:")
print(f"Точность: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"F1-macro: {f1_macro:.4f}")

print("\nКачество по классам:")
for class_name in ensemble.classes_:
    if class_name in report:
        f1 = report[class_name]['f1-score']
        precision = report[class_name]['precision']
        recall = report[class_name]['recall']
        print(f"  {class_name}: F1={f1:.3f}, Precision={precision:.3f}, Recall={recall:.3f}")

print("\nКлючевые улучшения:")
print("✓ Эмоциональные маркеры")
print("✓ Обработка отрицаний")
print("✓ Ансамбль из 4 моделей")
print("✓ Комбинирование векторизаторов")

if f1_macro > 0.7:
    print("\n✓ МОДЕЛЬ ГОТОВА К ИСПОЛЬЗОВАНИЮ")
    print("  Все классы распознаются с хорошим качеством")
else:
    print("\n⚠ Модель требует доработки")

print(f"\nРаспределение предсказаний:")
pred_distribution = pd.Series(y_pred).value_counts()
print(pred_distribution)


ФИНАЛЬНАЯ ОЦЕНКА:
Точность: 0.8929 (89.29%)
F1-macro: 0.8950

Качество по классам:
  negative: F1=0.943, Precision=1.000, Recall=0.893
  neutral: F1=0.887, Precision=0.940, Recall=0.839
  positive: F1=0.855, Precision=0.779, Recall=0.946

Ключевые улучшения:
✓ Эмоциональные маркеры
✓ Обработка отрицаний
✓ Ансамбль из 4 моделей
✓ Комбинирование векторизаторов

✓ МОДЕЛЬ ГОТОВА К ИСПОЛЬЗОВАНИЮ
  Все классы распознаются с хорошим качеством

Распределение предсказаний:
positive    68
neutral     50
negative    50
Name: count, dtype: int64


#### 21. Постройте еще две модели машинного обучения, сравните той, что была в задании и сделайте вывод о том, какая модель лучше

In [61]:
# Простая модель для сравнения (используем только TF-IDF)
simple_rf = RandomForestClassifier(random_state=42)
simple_rf.fit(X_train_tfidf, y_train)
simple_pred = simple_rf.predict(X_test_tfidf)
simple_accuracy = accuracy_score(y_test, simple_pred)

# Модель с балансировкой (используем только TF-IDF)
balanced_rf = RandomForestClassifier(class_weight='balanced', random_state=42)
balanced_rf.fit(X_train_tfidf, y_train)
balanced_pred = balanced_rf.predict(X_test_tfidf)
balanced_accuracy = accuracy_score(y_test, balanced_pred)

# Логистическая регрессия с балансировкой (используем комбинированные данные)
lr_balanced = LogisticRegression(
    C=2.0,
    class_weight='balanced',
    max_iter=2000,
    random_state=42
)
lr_balanced.fit(X_train_combined, y_train)
lr_pred = lr_balanced.predict(X_test_combined)
lr_accuracy = accuracy_score(y_test, lr_pred)

# Сравнение всех моделей
ensemble_accuracy = accuracy_score(y_test, y_pred)

print("СРАВНЕНИЕ МОДЕЛЕЙ:")
print("=" * 50)
print(f"1. Простая Random Forest: {simple_accuracy:.3f}")
print(f"2. Random Forest с балансировкой: {balanced_accuracy:.3f}")
print(f"3. Logistic Regression с балансировкой: {lr_accuracy:.3f}")
print(f"4. Ансамбль моделей: {ensemble_accuracy:.3f}")

# Определяем лучшую модель
accuracies = [simple_accuracy, balanced_accuracy, lr_accuracy, ensemble_accuracy]
model_names = ['Простая RF', 'RF с балансировкой', 'LR с балансировкой', 'Ансамбль']

best_idx = np.argmax(accuracies)
best_model = model_names[best_idx]
best_accuracy = accuracies[best_idx]

print(f"\nЛУЧШАЯ МОДЕЛЬ: {best_model}")
print(f"Точность: {best_accuracy:.3f}")
print(f"Улучшение: +{(best_accuracy - simple_accuracy)*100:.1f} п.п.")



СРАВНЕНИЕ МОДЕЛЕЙ:
1. Простая Random Forest: 0.976
2. Random Forest с балансировкой: 0.976
3. Logistic Regression с балансировкой: 0.976
4. Ансамбль моделей: 0.893

ЛУЧШАЯ МОДЕЛЬ: Простая RF
Точность: 0.976
Улучшение: +0.0 п.п.
